In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
df = spark.read.table("silver_fabpredictretail")
display(df)

In [ ]:
#spark.sql('drop table dimOrderdate_gold')  

In [ ]:
from pyspark.sql.types import *
from delta.tables import*
# Define the schema for the dimdate_gold table
DeltaTable.createIfNotExists(spark) \
     .tableName("dimOrderdate_gold") \
     .addColumn("OrderDate", DateType()) \
     .addColumn("Quarter", StringType()) \
     .addColumn("Day", IntegerType()) \
     .addColumn("Month", IntegerType()) \
     .addColumn("Year", IntegerType()) \
     .addColumn("mmmyyyy", StringType()) \
     .addColumn("yyyymm", StringType()) \
     .execute()

In [ ]:
#to create a dataframe for date dimension, dimdate_gold:
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.functions import col, dayofmonth, month, year, date_format, quarter,concat

## Add new column Quarter to dataframe
df = df.withColumn("quarter", concat( lit("Q"), quarter(col("Orderdate"))))


dfdimOrderDate_gold = df.dropDuplicates(["OrderDate"]).select(col("OrderDate"), \
         dayofmonth("OrderDate").alias("Day"), \
         col("quarter"), \
         month("OrderDate").alias("Month"), \
         year("OrderDate").alias("Year"), \
         date_format(col("OrderDate"), "MMM-yyyy").alias("mmmyyyy"), \
         date_format(col("OrderDate"), "yyyyMM").alias("yyyymm"), \
     ).orderBy("OrderDate")
#display(dfdimOrderDate_gold)

In [ ]:
from delta.tables import *
    
deltaTable = DeltaTable.forPath(spark, 'Tables/dimorderdate_gold')
 
dfUpdates = dfdimOrderDate_gold
    
deltaTable.alias('silver') \
   .merge(
     dfUpdates.alias('updates'),
     'silver.OrderDate = updates.OrderDate'
   ) \
    .whenMatchedUpdate(set =
     {
          
     }
   ) \
  .whenNotMatchedInsert(values =
     {
       "OrderDate": "updates.OrderDate",
       "Quarter": "updates.quarter",
       "Day": "updates.Day",
       "Month": "updates.Month",
       "Year": "updates.Year",
       "mmmyyyy": "updates.mmmyyyy",
       "yyyymm": "yyyymm"
     }
   ) \
   .execute()

In [ ]:
#spark.sql('drop table Dim_Customer_Gold')  

In [ ]:
# Define the schema for the Dim_Customer_Gold table
from pyspark.sql.types import *
from delta.tables import * 
DeltaTable.createIfNotExists(spark) \
    .tableName("Dim_Customer_Gold") \
    .addColumn("Customer_ID", IntegerType()) \
    .addColumn("Shipping_Address", StringType()) \
    .addColumn("City", StringType()) \
    .addColumn("State",  StringType()) \
    .addColumn("Country", StringType()) \
    .addColumn("ZipCode", IntegerType()) \
    .addColumn("Gender", StringType()) \
    .addColumn("Age", IntegerType()) \
    .addColumn("Customer_Type", StringType()) \
    .execute()

In [ ]:
#  drop duplicate policy rows and Create Dim_Customer_Gold dataframe
from pyspark.sql.functions import col, split

dfDimCustomer_silver = df.dropDuplicates(["Customer_ID","Shipping_Address","City","State","Country","Zipcode","Gender","Age","Customer_Type"])
#display(dfDimCustomer_silver)
 

In [ ]:
#Prepare the dataset to insert into Dim_Customer_Gold table
from pyspark.sql.functions import monotonically_increasing_id, col, when, coalesce, max, lit

dfDimCustomer_temp = spark.read.table("dim_customer_gold")

#MAXSno = dfDimCustomer_temp.select(coalesce(max(col("sno")),lit(0)).alias("MAXSno")).first()[0]

dfDimCustomer_gold = dfDimCustomer_silver.join(dfDimCustomer_temp, \
                    (dfDimCustomer_silver.Customer_ID == dfDimCustomer_temp.Customer_ID) & \
                    (dfDimCustomer_silver.Shipping_Address == dfDimCustomer_temp.Shipping_Address) & \
                    (dfDimCustomer_silver.City == dfDimCustomer_temp.City) &  \
                    (dfDimCustomer_silver.State == dfDimCustomer_temp.State) &  \
                    (dfDimCustomer_silver.Country == dfDimCustomer_temp.Country) & \
                    (dfDimCustomer_silver.Zipcode == dfDimCustomer_temp.ZipCode) & \
                    (dfDimCustomer_silver.Gender == dfDimCustomer_temp.Gender)  & \
                    (dfDimCustomer_silver.Age == dfDimCustomer_temp.Age) & \
                    (dfDimCustomer_silver.Customer_Type == dfDimCustomer_temp.Customer_Type) , "left_anti")

#dfDimCustomer_gold = dfDimCustomer_gold.withColumn("SNo",monotonically_increasing_id() + MAXSno + 1)

#display(dfDimCustomer_gold)

In [ ]:
#ensure that DimCustomer_Gold table remains up-to-date as new data comes in
from delta.tables import *

deltaTable = DeltaTable.forPath(spark,'Tables/dim_customer_gold')
dfMerge = dfDimCustomer_gold

deltaTable.alias('silver') \
    .merge(
        dfMerge.alias('updates'),
        'silver.Customer_ID = updates.Customer_ID AND silver.Shipping_Address = updates.Shipping_Address AND silver.City = updates.City AND silver.State = updates.State AND silver.Country = updates.Country AND silver.ZipCode = updates.Zipcode AND silver.Gender = updates.Gender AND silver.Age = updates.Age  AND silver.Customer_Type = updates.Customer_Type'
    ) \
    .whenMatchedUpdate(set =
        {
            "Customer_ID": "updates.Customer_ID",
            "Shipping_Address": "updates.Shipping_Address",
            "City": "updates.City",
            "State": "updates.State",
            "Country": "updates.Country",
            "ZipCode": "updates.Zipcode",
            "Gender": "updates.Gender",
            "Age": "updates.Age",
            "Customer_Type": "updates.Customer_Type"        
        }
    ) \
    .whenNotMatchedInsert(values = 
        {
            "Customer_ID": "updates.Customer_ID",
            "Shipping_Address": "updates.Shipping_Address",
            "City": "updates.City",
            "State": "updates.State",
            "Country": "updates.Country",
            "ZipCode": "updates.Zipcode",
            "Gender": "updates.Gender",
            "Age": "updates.Age",
            "Customer_Type": "updates.Customer_Type"   
        }
    ) \
    .execute()

In [ ]:
#spark.sql('drop table Dim_Product_Gold')  

In [ ]:
# Define the schema for the Dim_Product_Gold table
from pyspark.sql.types import *
from delta.tables import * 
DeltaTable.createIfNotExists(spark) \
    .tableName("Dim_Product_Gold") \
    .addColumn("Product_id", IntegerType()) \
    .addColumn("Product_Name", StringType()) \
    .addColumn("Product_Category", StringType()) \
    .execute()

In [ ]:
#  drop duplicate Product rows and Create dim_product_gold dataframe
from pyspark.sql.functions import col, split

dfDimProduct_silver = df.dropDuplicates(["Product_ID","Product_Name","Product_Category"])

In [ ]:
#Prepare the dataset to insert into dim_product_gold table
from pyspark.sql.functions import monotonically_increasing_id, col, when, coalesce, max, lit

dfDimProduct_temp = spark.read.table("dim_product_gold")

#MAXSno = dfDimProduct_temp.select(coalesce(max(col("sno")),lit(0)).alias("MAXSno")).first()[0]

dfDimProduct_gold = dfDimProduct_silver.join(dfDimProduct_temp, \
                    (dfDimProduct_silver.Product_ID == dfDimProduct_temp.Product_id) & \
                    (dfDimProduct_silver.Product_Name == dfDimProduct_temp.Product_Name) & \
                    (dfDimProduct_silver.Product_Category == dfDimProduct_temp.Product_Category), "left_anti")

#display(dfDimProduct_gold)
#dfDimProduct_gold = dfDimProduct_gold.withColumn("SNo",monotonically_increasing_id() + MAXSno + 1)


In [ ]:
#ensure that dim_Product_gold table remains up-to-date as new data comes in
from delta.tables import *

deltaTable = DeltaTable.forPath(spark,'Tables/dim_product_gold')
dfMerge = dfDimProduct_gold

deltaTable.alias('silver') \
    .merge(
        dfMerge.alias('updates'),
        'silver.Product_id = updates.Product_ID AND silver.Product_Name = updates.Product_Name  AND silver.Product_Category = updates.Product_Category'
    ) \
    .whenMatchedUpdate(set =
        {
            "Product_id": "updates.Product_ID",
            "Product_Name": "updates.Product_Name",
            "Product_Category": "updates.Product_Category"   
        }
    ) \
    .whenNotMatchedInsert(values = 
        {
            "Product_id": "updates.Product_ID",
            "Product_Name": "updates.Product_Name",
            "Product_Category": "updates.Product_Category"    
        }
    ) \
    .execute()

In [ ]:
#spark.sql('drop table dim_method_gold')  

In [ ]:
# Define the schema for the Dim_Method_Gold table
from pyspark.sql.types import *
from delta.tables import * 
DeltaTable.createIfNotExists(spark) \
    .tableName("Dim_Method_Gold") \
    .addColumn("Method_id", IntegerType()) \
    .addColumn("Payment_Method", StringType()) \
    .execute()

In [ ]:
#  drop duplicate Product rows and Create dim_Method_gold dataframe
from pyspark.sql.functions import col, split

dfDimMethod_silver = df.dropDuplicates(["Payment_Method"])
#display(dfDimMethod_silver)

In [ ]:
#Prepare the dataset to insert into DimMethod_Gold table
from pyspark.sql.functions import monotonically_increasing_id, col, when, coalesce, max, lit

dfDimMethod_temp = spark.read.table("Dim_Method_Gold")

MAXDimMethodID = dfDimMethod_temp.select(coalesce(max(col("Method_id")),lit(0)).alias("MAXDimMethodID")).first()[0]

dfDimMethod_Gold = dfDimMethod_silver.join(dfDimMethod_temp,(dfDimMethod_silver.Payment_Method == dfDimMethod_temp.Payment_Method) , "left_anti")
dfDimMethod_Gold = dfDimMethod_Gold.withColumn("Method_id",monotonically_increasing_id() + MAXDimMethodID + 1)
#display(dfDimMethod_Gold)


In [ ]:
#ensure that DimMethod_Gold table remains up-to-date as new data comes in
from delta.tables import *  

deltaTable = DeltaTable.forPath(spark,'Tables/dim_method_gold')

dfMerge = dfDimMethod_Gold

deltaTable.alias('silver') \
    .merge(
        dfMerge.alias('updates'),
        'silver.Payment_Method = updates.Payment_Method'
    ) \
    .whenMatchedUpdate(set =
        {
            "Method_id": "updates.Method_id",
            "Payment_Method": "updates.Payment_Method"
        }
    ) \
    .whenNotMatchedInsert(values = 
        {
            "Method_id": "updates.Method_id",
            "Payment_Method": "updates.Payment_Method"
        }
    ) \
    .execute()

In [ ]:
#spark.sql('drop table Dim_OrderStatus_Gold')  

In [ ]:
# Define the schema for the Dim_Method_Gold table
from pyspark.sql.types import *
from delta.tables import * 
DeltaTable.createIfNotExists(spark) \
    .tableName("Dim_OrderStatus_Gold") \
    .addColumn("OrderStatus_id", IntegerType()) \
    .addColumn("Order_Status", StringType()) \
    .execute()

In [ ]:
#  drop duplicate Product rows and Create dim_OrderSTatus_gold dataframe
from pyspark.sql.functions import col, split

dfDimOrderStatus_silver = df.dropDuplicates(["Order_Status"])
#display(dfDimOrderStatus_silver)


In [ ]:
#Prepare the dataset to insert into DimOrder_Status_Gold table
from pyspark.sql.functions import monotonically_increasing_id, col, when, coalesce, max, lit

dfDimOrderStatus_temp = spark.read.table("Dim_OrderStatus_Gold")

MAXDimOrderStatusID = dfDimOrderStatus_temp.select(coalesce(max(col("OrderStatus_id")),lit(0)).alias("MAXDimOrderStatusID")).first()[0]

dfDimOrderStatus_Gold = dfDimOrderStatus_silver.join(dfDimOrderStatus_temp,(dfDimOrderStatus_silver.Order_Status == dfDimOrderStatus_temp.Order_Status) , "left_anti")
dfDimOrderStatus_Gold = dfDimOrderStatus_Gold.withColumn("OrderStatus_id",monotonically_increasing_id() + MAXDimOrderStatusID + 1)
#display(dfDimOrderStatus_Gold)


In [ ]:
#ensure that DimOrder_Status_Gold table remains up-to-date as new data comes in
from delta.tables import *  

deltaTable = DeltaTable.forPath(spark,'Tables/dim_orderstatus_gold')

dfMerge = dfDimOrderStatus_Gold

deltaTable.alias('silver') \
    .merge(
        dfMerge.alias('updates'),
        'silver.Order_Status = updates.Order_Status'
    ) \
    .whenMatchedUpdate(set =
        {
            "OrderStatus_id": "updates.OrderStatus_id",
            "Order_Status": "updates.Order_Status"
        }
    ) \
    .whenNotMatchedInsert(values = 
        {
            "OrderStatus_id": "updates.OrderStatus_id",
            "Order_Status": "updates.Order_Status"
        }
    ) \
    .execute()

In [ ]:
#spark.sql('drop table fact_predictretail_gold')  

In [ ]:
# Define the schema for the Fact_PredictRetail_Gold table
from pyspark.sql.types import *
from delta.tables import * 
DeltaTable.createIfNotExists(spark) \
    .tableName("Fact_PredictRetail_Gold") \
    .addColumn("Order_Id", IntegerType()) \
    .addColumn("Customer_ID", IntegerType()) \
    .addColumn("Product_Id", IntegerType()) \
    .addColumn("Method_id", IntegerType()) \
    .addColumn("OrderStatus_id", IntegerType()) \
    .addColumn("OrderDate", DateType()) \
    .addColumn("Quantity",  ByteType()) \
    .addColumn("Price", FloatType()) \
    .addColumn("Total_Amount", FloatType()) \
    .addColumn("Review_Rating",  IntegerType()) \
    .execute()

In [ ]:
##Now create the Fact Table by combining all the dimension's fk 

from pyspark.sql.functions import col
 
dfdimcustomer_temp  = spark.read.table("dim_customer_gold")
dfdimmethod_temp  = spark.read.table("dim_method_gold")
dfdimorderstatus_temp  = spark.read.table("dim_orderstatus_gold")
dfdimProduct_temp  = spark.read.table("dim_product_gold") 
 

df_Fact_Retail_Gold = df.alias("df1").join(dfdimProduct_temp.alias("df2"),(df.Product_Name == dfdimProduct_temp.Product_Name) & (df.Product_Category == dfdimProduct_temp.Product_Category), "left") \
        .join(dfdimcustomer_temp.alias("df3"),(df.Shipping_Address == dfdimcustomer_temp.Shipping_Address) & (df.City == dfdimcustomer_temp.City) & (df.Country == dfdimcustomer_temp.Country) & (df.Zipcode == dfdimcustomer_temp.ZipCode) & (df.Gender == dfdimcustomer_temp.Gender) & (df.Age == dfdimcustomer_temp.Age) & (df.Customer_Type == dfdimcustomer_temp.Customer_Type), "left") \
        .join(dfdimmethod_temp.alias("df4"),(df.Payment_Method == dfdimmethod_temp.Payment_Method) , "left") \
        .join(dfdimorderstatus_temp.alias("df5"),(df.Order_Status == dfdimorderstatus_temp.Order_Status) , "left") \
    .select(col("df2.Product_ID"), \
        col("df3.Customer_ID"), \
        col("df4.Method_id"), \
        col("df5.OrderStatus_id"), \
        col("df1.Order_ID"), \
        col("df1.OrderDate"), \
        col("df1.Quantity"), \
        col("df1.Price"), \
        col("df1.Total_Amount"), \
        col("df1.Review_Rating") \
    ).orderBy(col("df1.OrderDate"))
    

display(df_Fact_Retail_Gold.head(10))

In [ ]:
from pyspark.sql.functions import col, split

df_Fact_Retail_Gold = df_Fact_Retail_Gold.dropDuplicates(["Customer_ID","Product_id","Order_ID","OrderStatus_id","Method_id","OrderDate","Quantity","Price","Total_Amount","Review_Rating"])
display(df_Fact_Retail_Gold.head(10))

In [ ]:
#merge fact_predictretail_gold
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, 'Tables/fact_predictretail_gold')

dfUpdates = df_Fact_Retail_Gold

deltaTable.alias('silver')\
    .merge(
        dfUpdates.alias('updates'),\
        'silver.Order_ID = updates.Order_ID AND silver.Customer_ID = updates.Customer_ID AND silver.Product_Id = updates.Product_Id'
        )\
    .whenMatchedUpdate(set = {
        })\
    .whenNotMatchedInsert(values =
        {
            "Order_ID": "updates.Order_ID",
            "Customer_ID": "updates.Customer_ID",
            "OrderDate": "updates.OrderDate",
            "Product_Id": "updates.Product_Id",
            "Method_id": "updates.Method_id",
            "OrderStatus_id": "updates.OrderStatus_id",
            "Quantity": "updates.Quantity",
            "Price": "updates.Price",
            "Total_Amount": "updates.Total_Amount",
            "Review_Rating": "updates.Review_Rating"
        })\
    .execute()